## Load libraries:

In [2]:
import boto3
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [3]:
%%bash

aws s3 ls s3://${bucket}/

2023-03-04 00:23:32 sagemaker-studio-328876407652-1zzlgzyu0f7
2023-03-04 00:28:42 sagemaker-us-east-1-328876407652
2023-03-18 04:47:03 testfinalproject088


In [4]:
# To list the S3 Bucket content
!aws s3 ls s3://sagemaker-us-east-1-993410942383/content-project/airline_data/

                           PRE cancellation_table/
2023-03-12 22:40:29       5158 Airline_Customer_Service_Commitments_for_Cancellation(1).csv
2023-03-12 22:40:29  941254313 WeatherEvents_Jan2016-Dec2021.csv
2023-03-12 22:40:29  162079876 WeatherUS_2016.csv
2023-03-12 22:40:29      60188 airlineID_carrier_table.csv
2023-03-12 22:40:32     312298 airport_codes_table.csv
2023-03-12 22:40:32  149933579 arline_flights_table.csv


## Set S3 Source Location (Public S3 Bucket):

In [5]:
s3_public_path = "s3://sagemaker-us-east-1-993410942383/content-project/airline_data/"

In [6]:
%store s3_public_path

Stored 's3_public_path' (str)


## Set S3 Destination Location (Our Private S3 Bucket):

In [7]:
s3_private_path = "s3://testfinalproject088/content/".format(bucket)
print(s3_private_path)

s3://testfinalproject088/content/


In [8]:
%store s3_private_path

Stored 's3_private_path' (str)


## List Files in our Private S3 Bucket in this Account:

## Create Athena Database:

In [11]:
# Import PyAthena
from pyathena import connect

ModuleNotFoundError: No module named 'pyathena'

In [ ]:
# Create Athena Database
database_name = "testanalysis"
table_name = "cancellationcode"

# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [ ]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [10]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)

NameError: name 'database_name' is not defined

In [54]:
# Verify The Database Has Been Created Succesfully
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,database_name
0,default
1,testanalysis


In [55]:
if database_name in df_show.values:
    ingest_create_athena_db_passed = True

In [56]:
%store ingest_create_athena_db_passed

Stored 'ingest_create_athena_db_passed' (bool)


In [57]:
%store -r s3_private_path

In [60]:
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS (
         code string,
         description string,
) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\\t' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')""".format(
    database_name, table_name, s3_private_path
)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS (
         code string,
         description string,
) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t' LINES TERMINATED BY '\n' LOCATION 'testanalysis'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')


In [58]:
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         code string,
         description string,
) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\\t' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')""".format(
    database_name, table_name, s3_private_path
)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS testanalysis.cancellationcode(
         code string,
         description string,
) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t' LINES TERMINATED BY '\n' LOCATION 's3://testfinalproject088/content/'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')


In [61]:
import pandas as pd

pd.read_sql(statement, conn)

Failed to execute query.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/pyathena/common.py", line 305, in _execute
    **request
  File "/opt/conda/lib/python3.7/site-packages/pyathena/util.py", line 84, in retry_api_call
    return retry(func, *args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/tenacity/__init__.py", line 379, in __call__
    do = self.iter(retry_state=retry_state)
  File "/opt/conda/lib/python3.7/site-packages/tenacity/__init__.py", line 314, in iter
    return fut.result()
  File "/opt/conda/lib/python3.7/concurrent/futures/_base.py", line 428, in result
    return self.__get_result()
  File "/opt/conda/lib/python3.7/concurrent/futures/_base.py", line 384, in __get_result
    raise self._exception
  File "/opt/conda/lib/python3.7/site-packages/tenacity/__init__.py", line 382, in __call__
    result = fn(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/botocore/client.py", line 530, in _api_call
    return

DatabaseError: Execution failed on sql: CREATE EXTERNAL TABLE IF NOT EXISTS (
         code string,
         description string,
) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t' LINES TERMINATED BY '\n' LOCATION 'testanalysis'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')
An error occurred (InvalidRequestException) when calling the StartQueryExecution operation: line 1:8: mismatched input 'EXTERNAL'. Expecting: 'OR', 'SCHEMA', 'TABLE', 'VIEW'
unable to rollback

## Visualize Dataset: 

In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [16]:
df = pd.read_csv("s3://sagemaker-us-east-1-993410942383/content-project/airline_data/airport_codes_table.csv")
df.head()

,Unnamed: 0,Code,Description
0,0,01A,"Afognak Lake, AK: Afognak Lake Airport"
1,1,03A,"Granite Mountain, AK: Bear Creek Mining Strip"
2,2,04A,"Lik, AK: Lik Mining Camp"
3,3,05A,"Little Squaw, AK: Little Squaw Airport"
4,4,06A,"Kizhuyak, AK: Kizhuyak Bay"


In [20]:
df2 = pd.read_csv("s3://sagemaker-us-east-1-993410942383/content-project/airline_data/cancellation_table/CancellationCode_table.csv")
df2.head()

,Unnamed: 0,Code,Description
0,0,A,Carrier
1,1,B,Weather
2,2,C,National Air System
3,3,D,Security


## Possible questions to explore includes:

1. What is the most common reason for flights being cancelled? 

### 1. What is the most common reason for flights being cancelled? 

In [ ]:
# SQL statement
statement = """
SELECT product_category, AVG(star_rating) AS avg_star_rating
FROM {}.{} 
GROUP BY product_category 
ORDER BY avg_star_rating DESC
""".format(
    database_name, table_name
)

print(statement)